In [1]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import UnstructuredURLLoader
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import TokenTextSplitter
import pickle
import os
import time
from apikey import apikey

In [2]:
os.environ["OPENAI_API_KEY"] = apikey

In [3]:
%%script echo skipping

# Read URLs from the file
with open('urls.txt', 'r') as file:
    urls = [line.strip() for line in file]

# Initialize the loader with the URLs
loaders = UnstructuredURLLoader(urls=urls)

# Load the data
data = loaders.load()

# Save data as a pickle file
with open('data.pkl', 'wb') as file:
    pickle.dump(data, file)

Couldn't find program: 'echo'


In [4]:
%%script echo skipping
# Load data from data.pkl
with open('data.pkl', 'rb') as file:
    data = pickle.load(file)

Couldn't find program: 'echo'


In [5]:
%%script echo skipping

# Text Splitter
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=100)

docs = text_splitter.split_documents(data)

print(len(docs))

Couldn't find program: 'echo'


In [6]:
# Create an instance of the OpenAIEmbeddings class 
embeddings = OpenAIEmbeddings()
embeddings

OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=None, openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=6, request_timeout=None, headers=None)

In [7]:
%%script echo skipping

# Embedding dataset in steps and saving them seperately
# Assuming docs is a list of documents
# Determine the step size
step = 10000

# Calculate how many iterations are needed
iterations = len(docs) // step + 1

# Iterate over the documents
for i in range(iterations):
    # Calculate start and end index
    start_index = i * step
    end_index = start_index + step

    # Check if end_index is out of range
    if end_index > len(docs):
        end_index = len(docs)

    # Subset the documents
    docs_to_process = docs[start_index:end_index]

    # Create FAISS database from the documents
    db = FAISS.from_documents(docs_to_process, embeddings)

    # Save the index
    db.save_local(f"index_{end_index}")

    # Wait for 10 seconds before the next iteration to prevent RateLimitError
    time.sleep(10)


Couldn't find program: 'echo'


In [8]:
%%script echo skipping

# Load each database 
db1 = FAISS.load_local("index_10000", embeddings)
db2 = FAISS.load_local("index_20000", embeddings)
.
.
.
db50 = db50 = FAISS.load_local("index_500000", embeddings)

Couldn't find program: 'echo'


In [9]:
%%script echo skipping

# Merge databases into a single file
for i in range(2, 51):
    exec(f'db1.merge_from(db{i})')

#print lenght of the merged database
print(len(db1.docstore._dict))

Couldn't find program: 'echo'


In [10]:
%%script echo skipping

# Save the final database to disc
db1.save_local("index")

Couldn't find program: 'echo'


In [7]:
# load database from the disc
db = FAISS.load_local("index_1000t", embeddings)

In [8]:
# Define llm = 'gpt-3.5-turbo'
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.1)
llm

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.1, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, openai_proxy=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

In [9]:
# Define memory instance
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer')

In [10]:
# Define AQ retrivial chain
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=db.as_retriever(), memory=memory)
chain

RetrievalQAWithSourcesChain(memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key='answer', input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history'), callbacks=None, callback_manager=None, verbose=False, combine_documents_chain=MapReduceDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:', template_format='f-string', validate_template=True), llm=ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'opena

In [11]:
chain({"question": "What are the application deadlines for undergraduate  courses for the upcoming academic year?"}, return_only_outputs=True)

{'answer': 'The application deadlines for undergraduate courses for the upcoming academic year are as follows:\n\n- 15 October: Deadline for applications for Cambridge, Oxford, or any course in medicine, dentistry, or veterinary/medical science. Applications should arrive at UCAS by 6pm.\n- 25 January: Main UCAS deadline for the majority of degree courses. The deadline is 6pm.\n- 24 March: Deadline for some art and design courses. However, other art and design courses will follow the 15 January deadline. It is recommended to check the UCAS website for specific deadlines.\n',
 'sources': 'https://www.aru.ac.uk/study/undergraduate/how-to-apply/application-timeline'}

In [12]:
chain({"question": "How can I apply for a Master's program?"}, return_only_outputs=True)

{'answer': "You can apply for a Master's program online. There is an application link on each course page. \n",
 'sources': '\n- https://www.aru.ac.uk/research/postgraduate-research/how-to-apply\n- https://www.aru.ac.uk/research/postgraduate-research/how-to-apply#skip-secondary-nav\n- https://www.aru.ac.uk/research/postgraduate-research/how-to-apply#start-main-nav\n- https://www.aru.ac.uk/research/postgraduate-research/how-to-apply#skip-main-nav'}

In [21]:
chain({"question": "What are the specific admission requirements for international students?"}, return_only_outputs=True)

{'answer': 'The specific admission requirements for international students include meeting the entry requirements for the course and immigration requirements, providing a personal bank statement or bank letter to confirm sufficient funds for tuition fees and living costs, submitting academic qualifications, obtaining necessary certificates such as a Police Registration Certificate and TB Screening Certificate, providing parental consent and birth certificate for students under 18 years of age, and obtaining an ATAS Clearance Certificate if stated in the offer letter/CAS. The maintenance amount required for living expenses varies depending on the length of the course and the location of study, but for study in Cambridge or Chelmsford for more than 9 months, the required maintenance amount is £1,023 for each month of the course or a maximum of £9,207 for 9 months. English language requirements for undergraduate courses at the university include a minimum IELTS score of 6.0 or equivalent.

In [22]:
chain({"question": "What documents do I need to submit when applying for a Master's program?"}, return_only_outputs=True)

{'answer': "When applying for a Master's program, you need to submit the following documents:\n- Qualification certificates and transcripts, including certified translations, where applicable\n- A personal statement\n- References/recommendation letters\n- Curriculum vitae/resume\n- Passport\n- Current and previous visa(s) (if applicable)\n- Proof of name change (if applicable)\n- Portfolio (if applicable)\n",
 'sources': '\n- https://www.aru.ac.uk/international/how-to-apply/step-2---submit-your-application-form#skip-main-nav\n- https://www.aru.ac.uk/research/postgraduate-research/how-to-apply'}

In [23]:
chain({"question": "Are there any scholarships available for international students?"}, return_only_outputs=True)

{'answer': "Yes, there are scholarships available for international students. Some of the scholarships available include the International Merit Scholarship, International Alumni Scholarship, International Early Payment Discount, Children's Book Illustration Inclusivity Bursary & Mentorship, British Council Scholarship for Women in STEM, and the GREAT Scholarship. International students can refer to the ARU website for more information on these scholarships.\n",
 'sources': '\n- https://www.aru.ac.uk/student-life/preparing-for-university/help-with-finances/scholarships#start-main-nav\n- https://www.aru.ac.uk/student-life/preparing-for-university/help-with-finances/scholarships/merit-bursary#skip-secondary-nav'}

In [24]:
chain({"question": "How can I learn more about the faculty members in the Department of Computer Science?"}, return_only_outputs=True)

{'answer': 'To learn more about the faculty members in the Department of Computer Science, you can visit the official website of Anglia Ruskin University (ARU). The ARU website provides information about the faculty members, including their expertise and contact information. Additionally, you can find a list of lecturers in the Department of Computer Science on the ARU website, which includes Mahdi Maktabdar Oghaz, Arooj Fatima, Razvan-Ioan Dinita, and George Wilson. Their profile pages and contact information are also provided on the website.\n',
 'sources': '\n- https://www.aru.ac.uk/about-us/find-an-expert?expertisearea=Computer+Science&researchsupervision=Yes\n- https://www.aru.ac.uk/study/undergraduate/computer-science-bsc#applynow_panel\n- https://www.aru.ac.uk/study/undergraduate/computer-science-bsc#key_facts\n- https://www.aru.ac.uk/study/undergraduate/computer-science-bsc#start-main-nav'}

In [26]:
chain({"question": "Could you provide more information about the study abroad programs offered by the university?"}, return_only_outputs=True)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


{'answer': 'The university offers study abroad programs through its ARU Global office. They have exchange partnerships with universities in Canada and the USA, including the University of New Brunswick, Flagler College, Marshall University, Ramapo College of New Jersey, SUNY Cortland, University of Illinois at Chicago, and Valparaiso University. Additionally, they have options for studying in other international destinations or with other universities through the ISEP program. The programs require additional costs for accommodation, living expenses, flights, visas, and health insurance. Application deadlines vary depending on the trimester. Studying abroad provides an opportunity for an unforgettable international experience, meeting people from all over the world, and developing skills for future employability. \n',
 'sources': '\n- https://www.aru.ac.uk/international/study-abroad/information-for-outgoing-aru-students/non-eu-exchanges#start-main-nav\n- https://www.aru.ac.uk/internatio

In [42]:
chain({"question": "How is the academic year structured? Are there semesters, quarters or terms?"}, return_only_outputs=True)

{'answer': 'The academic year is structured into trimesters.\n',
 'sources': 'https://www.aru.ac.uk/about-us/academic-calendar, https://www.aru.ac.uk/student-life/preparing-for-university/starting-your-course#start-main-nav'}

In [43]:
What undergraduate degree programs are currently offered at the university? Give me the total number and the names."}, return_only_outputs=True)chain({"question": "

{'answer': 'The university currently offers a total of 4 undergraduate degree programs. The names of the programs are BA (Hons) and Criminology and Sociology BA (Hons).\n',
 'sources': '\n- https://www.aru.ac.uk/study/subject-areas/social-sciences?modeofstudy=Full-time&page=2\n- https://www.aru.ac.uk/study/subject-areas/social-sciences?startdate=September&page=2\n- https://www.aru.ac.uk/study/subject-areas/social-sciences?startdate=September'}

In [44]:
chain({"question": "How many credits do you need to graduate from data science master course?"}, return_only_outputs=True)


{'answer': "I don't know the answer to the question about the number of credits required to graduate from a data science master course.\n",
 'sources': 'N/A'}

In [45]:
chain({"question": "What types of student housing are available at Anglia Ruskin University?"}, return_only_outputs=True)

{'answer': 'At Anglia Ruskin University, the types of student housing available include:\n- Student Village in Chelmsford, with flats of three, five, or six students, with own rooms and shared kitchens. Some rooms have en-suite shower facilities.\n- Private rented options in Peterborough, including small private halls of residence with en-suite shower facilities and shared housing.\n- Halls of residence on and off campus in Cambridge, as well as shared houses within two miles of the campus. Various room options are available, including rooms with shared bathroom facilities, rooms with en-suite shower facilities, and studios.\n- Anglia House in Cambridge.\n',
 'sources': '- https://www.aru.ac.uk/-/media/Files/student-life/accommodation/allocation-policy.pdf\n- https://www.aru.ac.uk/-/media/Files/International/pre-departure-guide-2021-22-planning-your-arrival.pdf\n- https://www.aru.ac.uk/student-life/accommodation/accommodation-in-cambridge\n- https://www.aru.ac.uk/student-life/accommoda

In [46]:
chain({"question": "Are there any clubs or societies I could join at Anglia Ruskin University?"}, return_only_outputs=True)

{'answer': 'Yes, there are over 100 student-led societies that you can join at Anglia Ruskin University. These societies range from Cheerleading to Chess, E-Sports to Engineering, and more. They are open to students from all courses and ages. You can find more information on the Anglia Ruskin University website.\n',
 'sources': '\n- https://www.aru.ac.uk/student-life/support-and-facilities/students-union-and-events#start-main-nav\n- https://www.aru.ac.uk/student-life/support-and-facilities/students-union-and-events#skip-main-nav\n- https://www.aru.ac.uk/student-life/support-and-facilities/students-union-and-events#skip-secondary-nav'}

In [47]:
chain({"question": "What athletic facilities does Anglia Ruskin University have?"}, return_only_outputs=True)

{'answer': 'There is no information available in the given document about the athletic facilities at Anglia Ruskin University.\n',
 'sources': '\n- https://www.aru.ac.uk/sport/sports-scholarships/high-performance-volleyball/prospective-students#start-main-nav\n- https://www.aru.ac.uk/study/course-search?modeofstudy=Degree+apprenticeship&location=Chelmsford\n- https://www.aru.ac.uk/study/subject-areas/art-and-design?location=Cambridge&page=1\n- https://www.aru.ac.uk/study/postgraduate/contemporary-ethics#course_introduction'}

In [48]:
chain({"question": "How can I get involved in student government at Anglia Ruskin University?"}, return_only_outputs=True)

{'answer': 'To get involved in student government at Anglia Ruskin University, you can become a sports committee member. You can speak to your club President for more information and to put yourself forward for the election. Committee members are elected annually at the club AGMs, usually in April, and positions include President, Captain, Kit and Equipment Officer, and Communications Officer.\n',
 'sources': 'https://www.aru.ac.uk/sport/opportunities-with-us#skip-main-nav, https://www.aru.ac.uk/sport/opportunities-with-us#skip-secondary-nav, https://www.aru.ac.uk/sport/opportunities-with-us, https://www.aru.ac.uk/sport/opportunities-with-us#start-main-nav'}

In [49]:
chain({"question": "Is Anglia Ruskin University ranked in the top 100 universities globally for computer science?"}, return_only_outputs=True)

{'answer': 'There is no information provided to determine whether Anglia Ruskin University is ranked in the top 100 universities globally for computer science.\nSOURCES:',
 'sources': ''}

In [50]:
chain({"question": "How can students with disabilities access accommodations?"}, return_only_outputs=True)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


{'answer': 'Students with disabilities can access accommodations through various means, such as seeking assistance from an adviser in the Disability or Dyslexia Service, or working with a specialist study skills tutor or mentor provided by DSA. \n',
 'sources': '\n- https://www.aru.ac.uk/student-life/support-and-facilities/study-skills/disability-support\n- https://www.aru.ac.uk/student-life/support-and-facilities/study-skills/disability-support#skip-secondary-nav\n- https://www.aru.ac.uk/student-life/support-and-facilities/study-skills/disability-support#start-main-nav\n- https://www.aru.ac.uk/student-life/support-and-facilities/study-skills/disability-support#skip-main-nav'}

In [51]:
chain({"question": "Is there a health center on campus? What services does it provide?"}, return_only_outputs=True)

{'answer': 'Yes, there is a health center on campus. The health center provides assistance with mental, physical, medical, and/or learning conditions through experienced wellbeing advisors and counsellors.\n',
 'sources': '\n- https://www.aru.ac.uk/study/undergraduate/business-and-healthcare-management-london#course_introduction\n- https://www.aru.ac.uk/study/undergraduate/business-and-healthcare-management-london#start-main-nav\n- https://www.aru.ac.uk/study/undergraduate/business-and-healthcare-management-london#teaching_assessment\n- https://www.aru.ac.uk/study/undergraduate/business-and-healthcare-management-london#careers'}

In [52]:
chain({"question": "Can you provide information on the transportation services available to students?"}, return_only_outputs=True)

{'answer': 'The transportation services available to students include bus services, drive alone options, and public transport discounts. The most popular mode of travel for students who live 4-10 miles from campus is the bus, while the most popular mode of travel for staff in the same distance range is drive alone. For staff and students who live 10-20 miles from campus, drive alone is the most popular mode of travel. The university provides up-to-date information on their website, social media platforms, and through their Anglia Ruskin app, which offers personalized timetables and other key services. Additionally, there are public transport discounts available, although only 18% of students are aware of them.\n',
 'sources': '\n- https://www.aru.ac.uk/-/media/Files/about-us/sustainability/2019-Travel_report.pdf\n- https://www.aru.ac.uk/study/open-day/your-undergraduate-open-day/open-day-checklist\n- https://www.aru.ac.uk/-/media/Files/about-us/Governance/aru-access-agreement17-18.pdf'

In [53]:
chain({"question": "Can you explain the process to report harassment or discrimination at the university?"}, return_only_outputs=True)

{'answer': "To report harassment or discrimination at the university, you can make a formal complaint by writing to the Office of the Secretary and Clerk at complaints@anglia.ac.uk. The university will conduct an investigation under the Staff Disciplinary Policy and Procedure, overseen by HR Services. Additionally, the university has a Harassment Support Network consisting of trained staff who can provide support and information to those who have experienced bullying, discrimination, harassment, hate crime, or sexual violence. The network includes trained Sexual Violence Liaison Officers (SVLOs). The university's Rules, Regulations, and Procedures for Students explicitly state that harassment, bullying, discrimination, hate crimes, physical violence, sexual harassment, and violence, as well as victimization, are unacceptable forms of misconduct. The Code of Conduct for Students emphasizes the importance of professionalism and prohibits abusive, threatening, or offensive language or act

In [54]:
chain({"question": "Are there any mentorship programs or peer-tutoring services at the university?"}, return_only_outputs=True)

{'answer': 'Yes, there are mentorship programs and peer-tutoring services at the university. The university offers a Peer Wellbeing Mentor program where students can receive support from their peers through shared experiences. Additionally, there is a mentoring program at ARU that matches current students with mentors on a 1-2-1 basis. Students and mentors can apply to join the program through an online application form.\n',
 'sources': '\n- https://www.aru.ac.uk/current-students/health-and-wellbeing/look-after-yourself/peer-wellbeing-mentors\n- https://www.aru.ac.uk/business-employers/access-student-and-graduate-talent/get-involved-in-supporting-our-students/mentoring'}

In [55]:
chain({"question": "How is the library facilities at Anglia Ruskin University?"}, return_only_outputs=True)

{'answer': 'The library facilities at Anglia Ruskin University are well-equipped with a significant collection of course-relevant materials, study spaces including group work zones and silent study areas, multimedia playback, networked PCs, wireless connectivity, and standard IT applications. The libraries in Cambridge and Chelmsford are open 24 hours during main assessment periods, while the library in Peterborough, although smaller, provides spaces for quiet individual study and networked PCs. Students also have access to e-books and other online resources for their courses. \n',
 'sources': 'https://www.aru.ac.uk/student-life/support-and-facilities/world-class-facilities/library-and-it'}

In [56]:
chain({"question": "Is it correct that the university's library is open 24/7?"}, return_only_outputs=True)

{'answer': "Yes, it is correct that the university's library is open 24/7 during main assessment periods.\n",
 'sources': 'https://www.aru.ac.uk/student-life/support-and-facilities/world-class-facilities/library-and-it#start-main-nav'}

In [57]:
chain({"question": "Can an alumni use the library after graduating?"}, return_only_outputs=True)

{'answer': 'Yes, alumni can use the library after graduating. They can apply for borrowing rights or reference access to the libraries. Access is free for the first year after graduation, and after that, there is an annual fee for borrowing membership. Reference access is free if they have a library card. \n',
 'sources': 'https://www.aru.ac.uk/graduation-and-alumni/alumni-network/benefits-and-services'}

In [58]:
chain({"question": "What are the opening hours of the main library?"}, return_only_outputs=True)

{'answer': "The main library's opening hours are 24 hours during main assessment periods.\n",
 'sources': 'https://www.aru.ac.uk/student-life/support-and-facilities/world-class-facilities/library-and-it#start-main-nav, https://www.aru.ac.uk/student-life/support-and-facilities/world-class-facilities/library-and-it#skip-main-nav, https://www.aru.ac.uk/student-life/support-and-facilities/world-class-facilities/library-and-it#skip-secondary-nav, https://www.aru.ac.uk/student-life/support-and-facilities/world-class-facilities/library-and-it'}

In [59]:
chain({"question": "Can students access the library's digital resources remotely?"}, return_only_outputs=True)

{'answer': "Yes, students can access the library's digital resources remotely.\n",
 'sources': 'https://www.aru.ac.uk/student-life/support-and-facilities/world-class-facilities/library-and-it#start-main-nav, https://www.aru.ac.uk/student-life/support-and-facilities/world-class-facilities/library-and-it#skip-main-nav, https://www.aru.ac.uk/student-life/support-and-facilities/world-class-facilities/library-and-it#skip-secondary-nav'}

In [60]:
chain({"question": "Does the university offer work-study programs?"}, return_only_outputs=True)

{'answer': 'The university does not offer work-study programs.\n',
 'sources': 'https://www.aru.ac.uk/arts-humanities-and-social-sciences/creative-industries/why-study-with-us/placements-and-work-experience/work-placement-faqs, https://www.aru.ac.uk/study/course-search?keywords=&levelofstudy=&levelofstudy=&location=University+Centre+West+Anglia&modeofstudy=&faculty=&availability='}

In [61]:
chain({"question": "What is the deadline to apply for financial aid for the upcoming academic year?"}, return_only_outputs=True)

{'answer': 'The deadline to apply for financial aid for the upcoming academic year is not provided in the given information.\n',
 'sources': 'N/A'}

In [62]:
chain({"question": "What criteria does the university consider when awarding scholarships?"}, return_only_outputs=True)

{'answer': "The university considers various criteria when awarding scholarships, including financial need, academic progress, and specific criteria for different scholarships such as the International Excellence Scholarship. The specific criteria can be found in the handbook and on the university's website.\n",
 'sources': '\n- https://www.aru.ac.uk/-/media/Files/about-us/sustainability/Society-Green-Fund-Handbook.pdf\n- https://www.aru.ac.uk/-/media/Files/student-life/eligibility-british-council-scholarship-women-in-stem.pdf\n- https://www.aru.ac.uk/student-life/preparing-for-university/help-with-finances/scholarships#start-main-nav\n- https://www.aru.ac.uk/student-life/preparing-for-university/help-with-finances/scholarships'}

In [63]:
chain({"question": "What types of financial aid are available to students?"}, return_only_outputs=True)

{'answer': "The types of financial aid available to students include graduate loans, grants and allowances, Disabled Students' Allowance (DSA), tax credits, Maintenance Loan, scholarships, bursaries, and US Federal Financial Aid (student loans).\n",
 'sources': '\n- https://www.aru.ac.uk/student-life/preparing-for-university/help-with-finances/postgraduate#skip-main-nav\n- https://www.aru.ac.uk/student-life/preparing-for-university/help-with-finances/undergraduate\n- https://www.aru.ac.uk/international/information-by-country/united-states/us-loans/disclaimer#skip-secondary-nav\n- https://www.aru.ac.uk/international/information-by-country/united-states/us-loans/disclaimer#skip-main-nav'}

In [64]:
chain({"question": "What is the process to apply for financial aid?"}, return_only_outputs=True)

{'answer': 'To apply for financial aid, the first step is to complete a Free Application for Federal Student Aid (FAFSA). This will determine your eligibility for a loan. The School Code for Anglia Ruskin University is G34353. If you are applying for a PLUS loan or a Parent PLUS loan, you will need to complete a PLUS loan request on StudentLoans.gov. The US Department of Education may also require you to complete an online PLUS loan counseling module. Once your FAFSA has been completed and processed, you will receive a Student Aid Report (SAR) which lists the information you provided and your Expected Family Contribution (EFC). The EFC is used to determine eligibility for needs-based financial aid. It is important to review the SAR for accuracy. \n',
 'sources': '\n- https://www.aru.ac.uk/international/information-by-country/united-states/us-loans/how-to-apply#start-main-nav\n- https://www.aru.ac.uk/international/information-by-country/united-states/us-loans/how-to-apply#skip-secondary

In [65]:
chain({"question": "Are there any networking events or reunions for alumni?"}, return_only_outputs=True)

{'answer': 'Yes, there are networking events and reunions for alumni. The university can help plan a reunion back on campus, including delivering a tour of the campus and assisting in tracking down lost friends to join the event.\n',
 'sources': '\n- https://www.aru.ac.uk/graduation-and-alumni/alumni-network/staying-in-touch/find-a-friend\n- https://www.aru.ac.uk/graduation-and-alumni/alumni-network/staying-in-touch/find-a-friend#skip-main-nav\n- https://www.aru.ac.uk/graduation-and-alumni/alumni-network/staying-in-touch/find-a-friend#start-main-nav\n- https://www.aru.ac.uk/graduation-and-alumni/alumni-network/staying-in-touch/find-a-friend#skip-secondary-nav'}

In [66]:
chain({"question": "Are there any benefits or discounts available to alumni?"}, return_only_outputs=True)

{'answer': 'Yes, there are benefits and discounts available to alumni. These include free careers support, discounts on postgraduate study, access to University facilities such as gyms and libraries, and discounts on room hire and rent in Innovation Centres. Alumni can also apply for borrowing rights or reference access to the libraries. The specific discounts and benefits may vary, so it is recommended to visit the official website for more information.\n',
 'sources': '\n- https://www.aru.ac.uk/graduation-and-alumni/alumni-network/benefits-and-services\n- https://www.aru.ac.uk/graduation-and-alumni/alumni-network/benefits-and-services#skip-secondary-nav\n- https://www.aru.ac.uk/graduation-and-alumni/alumni-network/benefits-and-services#start-main-nav'}

In [67]:
chain({"question": "How can alumni stay involved with university clubs or organizations post-graduation?"}, return_only_outputs=True)

{'answer': "Alumni can stay involved with university clubs or organizations post-graduation by volunteering through the ARU Students' Union Volunteering Service. They can arrange a one-to-one meeting with a Students' Union Volunteer Coordinator to get started. Additionally, alumni can become International Alumni Ambassadors, representing the university in their home country and helping to maintain the relationship with international alumni. They can also get involved by speaking to student groups, participating in employability events, or giving guest lectures. For more information, alumni can find contact details online or email [email\xa0protected] or [email\xa0protected].\n",
 'sources': '\n- https://www.aru.ac.uk/graduation-and-alumni/alumni-network/get-involved#start-main-nav\n- https://www.aru.ac.uk/graduation-and-alumni/alumni-network/get-involved#skip-secondary-nav\n- https://www.aru.ac.uk/graduation-and-alumni/alumni-network/get-involved\n- https://www.aru.ac.uk/graduation-and

In [68]:
chain({"question": "Can alumni access the university's career services?"}, return_only_outputs=True)

{'answer': "Yes, alumni can access the university's career services. They can access the Alumni Career Centre by emailing Viktoriya Vladimirova, the Graduate Employability & Careers Adviser, and providing their full name and degree.\n",
 'sources': '\n- https://www.aru.ac.uk/-/media/Files/alumni-and-supporters/connect-may-2022.pdf\n- https://www.aru.ac.uk/graduation-and-alumni/achieve-your-career-goals#skip-main-nav\n- https://www.aru.ac.uk/graduation-and-alumni/achieve-your-career-goals#start-main-nav\n- https://www.aru.ac.uk/graduation-and-alumni/achieve-your-career-goals'}

In [69]:
chain({"question": "How can alumni stay updated about the latest news and developments at the university?"}, return_only_outputs=True)

{'answer': 'Alumni can stay updated about the latest news and developments at the university by staying in touch, attending reunions, networking events, and lectures. They can also receive publications such as alumni magazines and updates about the university.\n',
 'sources': '\n- https://www.aru.ac.uk/graduation-and-alumni/alumni-network#skip-main-nav\n- https://www.aru.ac.uk/privacy-and-cookies/alumni-supporters#start-main-nav'}

In [70]:
chain({"question": "What is the total student population of the university?"}, return_only_outputs=True)

{'answer': 'There is no information available in the given document to determine the total student population of the university.\n',
 'sources': '\n- https://www.aru.ac.uk/-/media/Files/about-us/working-here/faculty-of-science-and-engineering-athena-swan-bronze-award-submission.pdf\n- https://www.aru.ac.uk/-/media/Files/research/HR-Excellence-Awards/FINAL-HR-EiR-Forward-looking-action-plan-2021_2023_20_5_21.xlsx\n- https://www.aru.ac.uk/-/media/Files/research/HR-Excellence-Awards/2023-HREIR-Award-Backward-Looking-Action-Plan-final-for-submission.xlsx\n- https://www.aru.ac.uk/-/media/Files/about-us/equality-diversity-and-inclusion/EDI-Annual-Report-2014--2015.pdf'}

In [71]:
chain({"question": "How many campuses does the university have and where are they located?"}, return_only_outputs=True)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


{'answer': 'The university has multiple campuses, including Cambridge, Chelmsford, ARU Peterborough, ARU London, and University Centre West Anglia. The exact number of campuses is not specified.\n',
 'sources': 'https://www.aru.ac.uk/study/explore-aru/teaching-excellence/staff-teaching-qualifications#start-main-nav, https://www.aru.ac.uk/study/explore-aru/teaching-excellence/staff-teaching-qualifications, https://www.aru.ac.uk/study/explore-aru/teaching-excellence/staff-teaching-qualifications?page=3, https://www.aru.ac.uk/study/undergraduate/audio-and-music-technology#skip-main-nav'}

In [72]:
chain({"question": "What are some of the notable achievements or awards of the university?"}, return_only_outputs=True)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


{'answer': "Some of the notable achievements or awards of the university include:\n\n- Alumni Voluntary Service Award: Winner - Mary Stretch (PGC Education (Secondary) 2001)\n- Alumni Public Service Award: Winner - Mark Songhurst (BA (Hons) European Business 1999)\n- Alumni Rising Star Award: Winner - Aining Wen (MA in Children's Book Illustration 2017)\n- Alumni Contribution to Culture Award: Winner - Joseph A Simpson (BSc (Hons) Audio & Music Technology 2012)\n- Alumni Entrepreneur of the Year Award\n",
 'sources': 'https://www.aru.ac.uk/graduation-and-alumni/alumni-network/alumni-awards-home/alumni-awards-2023-winners'}

In [73]:
chain({"question": "What facilities and resources are available to students at the university?"}, return_only_outputs=True)

{'answer': 'The facilities and resources available to students at the university include:\n- Placement support for gaining work experience and industry contacts\n- Chaplaincy services\n- Sports and leisure facilities\n- Arts facilities, including the Ruskin Gallery\n- University library resources\n- Conference venue hire and access to specialist services and lab spaces\n',
 'sources': 'https://www.aru.ac.uk/about-us/governance/policies-procedures-and-regulations/access-to-information/publication-scheme#skip-secondary-nav, https://www.aru.ac.uk/about-us/governance/policies-procedures-and-regulations/access-to-information/publication-scheme#skip-main-nav, https://www.aru.ac.uk/about-us/governance/policies-procedures-and-regulations/access-to-information/publication-scheme#start-main-nav'}

In [74]:
chain({"question": "What sustainability practices does the university follow?"}, return_only_outputs=True)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


{'answer': 'The university follows several sustainability practices, including:\n\n1. Championing sustainability through student societies and regular events on campuses.\n2. Addressing sustainability in research ethics procedures.\n3. Developing attractive and efficient campuses that showcase commitment to sustainability.\n4. Providing practical opportunities for environmental learning, research, and socializing on campuses.\n5. Meeting the international standard for environmental management (ISO 14001) in building maintenance and operations.\n6. Strengthening resilience against environmental risks through risk management processes.\n7. Applying sustainability criteria in procurement, including food provision.\n8. Sharing learning and collaborating with local and regional communities.\n9. Integrating sustainability into the curriculum, research work, and campuses.\n10. Encouraging and supporting student initiatives.\n11. Being certified to ISO 14001 since 2009 to ensure compliance wit

In [75]:
chain({"question": "What is the address of the Cambridge campus of the university?"}, return_only_outputs=True)

{'answer': 'The address of the Cambridge campus of the university is East Road, Cambridge CB1 1PT.\n',
 'sources': '\n- https://www.aru.ac.uk/-/media/Files/study-with-us/open-days/cambridge-open-day-map.pdf\n- https://www.aru.ac.uk/-/media/Files/about-us/sustainability/FINAL-PRME-report.pdf'}

In [76]:
chain({"question": "How can I contact the admissions office of the university?"}, return_only_outputs=True)

{'answer': 'To contact the admissions office of the university, you should reach out to your Admissions faculty team. They can provide advice and assistance. \n',
 'sources': '\n- https://www.aru.ac.uk/study/admissions/after-you-apply#start-main-nav\n- https://www.aru.ac.uk/study/admissions/after-you-apply\n- https://www.aru.ac.uk/study/admissions/after-you-apply#skip-main-nav\n- https://www.aru.ac.uk/study/admissions/after-you-apply#skip-secondary-nav'}

In [77]:
chain({"question": "Where can I find information about parking on campus?"}, return_only_outputs=True)

{'answer': 'Information about parking on campus can be found on the ARU website at the following link: https://www.aru.ac.uk/business-employers/develop-your-business/conference-meeting-room-hire/frequently-asked-questions#skip-main-nav\n',
 'sources': 'https://www.aru.ac.uk/business-employers/develop-your-business/conference-meeting-room-hire/frequently-asked-questions#skip-main-nav'}

In [78]:
chain({"question": "Who do I contact about transcript requests?"}, return_only_outputs=True)

{'answer': "If you'd like a copy of your transcript or degree certificate and you graduated in 1992 or after, you can visit the online store of the institution. You will need to know your student ID. If you are unsure of your student ID or if you completed your studies before 1992, you can contact the awards team via email at [email protected]\n",
 'sources': 'https://www.aru.ac.uk/graduation-and-alumni/alumni-network/staying-in-touch/transcripts-certificates-and-reference-requests#skip-secondary-nav'}

In [79]:
chain({"question": "Is there a hotline for reporting issues related to campus maintenance?"}, return_only_outputs=True)

{'answer': 'To report non-urgent repairs related to campus maintenance, you can email [email protected] or call 01245 686464.\n',
 'sources': '\n- https://www.aru.ac.uk/-/media/Files/student-life/accommodation/current-students/Management-plans-and-code-of-practice/Management-plans.pdf\n- https://www.aru.ac.uk/student-life/accommodation/current-students/who-to-ask#village\n- https://www.aru.ac.uk/student-life/accommodation/current-students/who-to-ask#collier\n- https://www.aru.ac.uk/student-life/accommodation/current-students/who-to-ask#uni'}

In [80]:
chain({"question": "What career services does the university offer to its students?"}, return_only_outputs=True)

{'answer': 'The university offers a dedicated careers and employability service that provides advice and guidance to students. They offer careers advice, personalized appointments with experienced advisers, help with CVs, job searches, applications, and interview preparation. They also have an online Employability Programme to help students develop the skills that employers are looking for. Additionally, they provide support for finding jobs, work experience, start-up support, and language learning resources.\n',
 'sources': '\n- https://www.aru.ac.uk/student-life/support-and-facilities/careers-and-employability#skip-main-nav\n- https://www.aru.ac.uk/student-life/support-and-facilities/careers-and-employability#start-main-nav\n- https://www.aru.ac.uk/student-life/support-and-facilities/careers-and-employability#skip-secondary-nav\n- https://www.aru.ac.uk/student-life/support-and-facilities/careers-and-employability'}

In [81]:
chain({"question": "Does the university offer internships or co-op opportunities? Give some examples."}, return_only_outputs=True)

{'answer': 'The university does not offer internships or co-op opportunities. However, students are encouraged to find placements themselves, which can be anywhere in the UK or even internationally. Previous placements have been in locations such as Cambridge, London, Gibraltar, Hong Kong, and Taiwan.\n',
 'sources': '\n- https://www.aru.ac.uk/arts-humanities-and-social-sciences/creative-industries/why-study-with-us/placements-and-work-experience/work-placement-faqs\n- https://www.aru.ac.uk/arts-humanities-and-social-sciences/cambridge-school-of-art/why-study-with-us/placements-and-work-experience/placement-faqs#where-can-i-work\n- https://www.aru.ac.uk/arts-humanities-and-social-sciences/cambridge-school-of-art/why-study-with-us/placements-and-work-experience/placement-faqs#start-main-nav\n- https://www.aru.ac.uk/arts-humanities-and-social-sciences/cambridge-school-of-art/why-study-with-us/placements-and-work-experience/placement-faqs#when-placement-happens'}

In [82]:
chain({"question": "Are there any job fairs hosted by the university throughout the year?"}, return_only_outputs=True)

{'answer': 'There is no information available in the given document regarding job fairs hosted by the university.\n',
 'sources': '\n- https://www.aru.ac.uk/student-life/accommodation/private-rented-accommodation/house-hunting-events\n- https://www.aru.ac.uk/student-life/accommodation/private-rented-accommodation/house-hunting-events#skip-secondary-nav\n- https://www.aru.ac.uk/student-life/accommodation/private-rented-accommodation/house-hunting-events#start-main-nav\n- https://www.aru.ac.uk/student-life/accommodation/private-rented-accommodation/house-hunting-events#skip-main-nav'}

In [83]:
chain({"question": "How does the university support student entrepreneurs or startups?"}, return_only_outputs=True)

{'answer': "The university supports student entrepreneurs or startups by providing business support and mentoring through their start-up hub. Additionally, students have the opportunity to pitch for start-up funding in the university's annual business competition.\n",
 'sources': 'https://www.aru.ac.uk/study/subject-areas/business?levelofstudy=Postgraduate&startdate=January#coursesearch'}

In [84]:
chain({"question": "Does the university offer any part-time jobs to their students? If yes, give some details."}, return_only_outputs=True)

{'answer': 'Yes, the university offers part-time jobs to their students. Unfortunately, there are no specific details provided about these part-time jobs.\n',
 'sources': '\n- https://www.aru.ac.uk/study/subject-areas/accounting-and-finance?startdate=September&modeofstudy=Part-time#coursesearch\n- https://www.aru.ac.uk/study/subject-areas/accounting-and-finance?startdate=September&modeofstudy=Part-time#skip-main-nav\n- https://www.aru.ac.uk/study/subject-areas/accounting-and-finance?startdate=September&modeofstudy=Part-time\n- https://www.aru.ac.uk/study/subject-areas/accounting-and-finance?startdate=September&modeofstudy=Part-time#start-main-nav'}

In [85]:
chain({"question": "Where can I find a calendar of upcoming events on campus?"}, return_only_outputs=True)

{'answer': 'You can find a calendar of upcoming events on campus on the ARU website. The academic calendar contains information on key dates for the academic year, including teaching weeks, examination periods, and vacations for all taught courses. Additionally, there are talks, workshops, exhibitions, performances, and family events that are open to everyone. You can browse upcoming events on the ARU website.\n',
 'sources': '\n- https://www.aru.ac.uk/student-life/preparing-for-university/starting-your-course#start-main-nav\n- https://www.aru.ac.uk/study/open-day#skip-main-nav'}

In [86]:
chain({"question": "How can I organize an event on campus?"}, return_only_outputs=True)

{'answer': "To organize an event on campus, you can use the Announcements and Events sections on My ARU, infoscreens around campus, and posters. However, there are strict rules about posters at ARU, so it's important to contact the relevant department for guidance. If you plan to invite an external speaker, you must submit a Guest Speaker Form to the relevant department or the Student Activities Department of the Students' Union four weeks in advance. The form will be assessed, and a decision will be communicated to the society committee responsible for the booking. Four weeks' notice is required for the necessary assessments and decision-making. \n",
 'sources': '\n- https://www.aru.ac.uk/international-community-experience/fundraising-ideas\n- https://www.aru.ac.uk/about-us/equality-diversity-and-inclusion/freedom-of-speech-and-prevent#skip-secondary-nav\n- https://www.aru.ac.uk/about-us/equality-diversity-and-inclusion/freedom-of-speech-and-prevent#skip-main-nav'}

In [87]:
chain({"question": "What cultural or diversity-related events are offered at the university?"}, return_only_outputs=True)

{'answer': 'There is no information provided in the given document about the cultural or diversity-related events offered at the university.\n',
 'sources': 'https://www.aru.ac.uk/study/open-day#skip-main-nav, https://www.aru.ac.uk/study/open-day, https://www.aru.ac.uk/anglia-learning-and-teaching/cpd-opportunities, https://www.aru.ac.uk/anglia-learning-and-teaching/cpd-opportunities#start-main-nav'}

In [89]:
chain({"question": "Is there a commencement ceremony for graduating students? If yes, give more information about when and where it happens."}, return_only_outputs=True)

{'answer': 'Yes, there is a commencement ceremony for graduating students. After the close of proceedings, all graduates are asked to process out, led by the platform party. They will process around the cathedral and then attend a post-ceremony drinks reception in the marquee.\n',
 'sources': '\n- https://www.aru.ac.uk/graduation-and-alumni/your-graduation-ceremony/itineraries/peterborough-graduations#skip-main-nav\n- https://www.aru.ac.uk/graduation-and-alumni/your-graduation-ceremony/itineraries/peterborough-graduations#start-main-nav\n- https://www.aru.ac.uk/graduation-and-alumni/your-graduation-ceremony/itineraries/peterborough-graduations#skip-secondary-nav'}

In [90]:
chain({"question": "Are there any networking events or career fairs happening this semester?"}, return_only_outputs=True)

{'answer': 'There is no specific information provided about networking events or career fairs happening this semester.\nSOURCES:',
 'sources': ''}

In [91]:
chain({"question": "Where can I find the latest news about the university?"}, return_only_outputs=True)

{'answer': 'You can find the latest news about the university on their website at the following links:\n- Report: https://www.aru.ac.uk/study/undergraduate/animal-behaviour#accreditation\n- Report: https://www.aru.ac.uk/study/undergraduate/animal-behaviour#course_introduction\n- Report: https://www.aru.ac.uk/study/undergraduate/animal-behaviour#course_options\n- Report: https://www.aru.ac.uk/study/undergraduate/animal-behaviour#lectures\n',
 'sources': '\n- https://www.aru.ac.uk/study/undergraduate/animal-behaviour#accreditation\n- https://www.aru.ac.uk/study/undergraduate/animal-behaviour#course_introduction\n- https://www.aru.ac.uk/study/undergraduate/animal-behaviour#course_options\n- https://www.aru.ac.uk/study/undergraduate/animal-behaviour#lectures'}

In [92]:
chain({"question": "How can I contribute news articles or announcements to the university's news platform?"}, return_only_outputs=True)

{'answer': "To contribute news articles or announcements to the university's news platform, you can send an email to the editor, Nickie Efford-Eliraz, at se-first@anglia.ac.uk.\n",
 'sources': 'https://www.aru.ac.uk/-/media/Files/Faculty/FSE/Research/First-October-2019.pdf, https://www.aru.ac.uk/-/media/Files/Faculty/FSE/Research/First---Winter-2020.pdf, https://www.aru.ac.uk/science-and-engineering/research/research-news#skip-secondary-nav, https://www.aru.ac.uk/science-and-engineering/research/research-news#start-main-nav'}

In [93]:
chain({"question": "What is the university's recent stance on sustainability and climate change?"}, return_only_outputs=True)

{'answer': 'The university is committed to sustainability and addressing climate change. They have a Sustainability Strategy in place, which includes goals to become a zero-carbon university by 2045 and reduce direct carbon emissions. They have implemented various measures across their operations, including policies on agile working, education provision, estates management, investment strategy, procurement and purchasing decisions, water and waste management, and travel. They are also a signatory to the Fossil Fuel Declaration, committing to no investments in extractor fossil fuel companies and remaining fossil-free in the future. The university aims to embed sustainability in all aspects of its activities, including teaching, research, operations, and partnerships. They strive to equip their graduates with the knowledge, skills, and agency to address climate and ecological emergencies and work alongside local communities to promote sustainability. \n',
 'sources': '\n- https://www.aru

In [94]:
chain({"question": "What are the latest updates on the university's expansion or development plans?"}, return_only_outputs=True)

{'answer': 'The university has plans for expansion, with a focus on developing a masterplan that takes into account stakeholder engagement. The development projects are being planned, procured, constructed, or recently commissioned across the three ARU owned campuses. The university has received positive feedback from stakeholders and students regarding the plans. The development of the masterplan will follow the right process, with each individual capital project supported by a robust business case. Updates on the progress of the development plans will be provided at future Estates Committee meetings. The current building is constructed to accommodate 500 students, which is deemed sufficient for now. The university is also starting a fifth cohort of undergraduate medical students in September. Consideration has been given to the siting of the building in relation to potential future development and site links.\n',
 'sources': '1. https://www.aru.ac.uk/-/media/Files/about-us/Governance

In [95]:
chain({"question": "Where can I find news about upcoming academic conferences or seminars?"}, return_only_outputs=True)

{'answer': "You can find news about upcoming academic conferences or seminars on the conference website of the Research on Architecture Conference hosted by ArchiDOCT and ARU's Architecture and Planning team. The call for papers is currently open. Here is the link to the conference website: [source]\n",
 'sources': 'https://www.aru.ac.uk/science-and-engineering/conferences-and-events/research-on-architecture-conference#start-main-nav'}

In [96]:
chain({"question": "What are the visa requirements for international students wishing to study at ARU?"}, return_only_outputs=True)

{'answer': 'There is no relevant information provided in the given portion of the document to answer the question about visa requirements for international students wishing to study at ARU. For accurate and up-to-date information on visa requirements, it is recommended to visit the official website of ARU or contact the university directly.\n',
 'sources': '\n- https://www.aru.ac.uk/international/information-by-country/united-states\n- https://www.aru.ac.uk/international/how-to-apply#skip-main-nav'}

In [97]:
chain({"question": "Are there any scholarships or financial aid options available specifically for international students?"}, return_only_outputs=True)

{'answer': 'There is no specific mention of scholarships or financial aid options available specifically for international students.\n',
 'sources': 'https://www.aru.ac.uk/study/postgraduate/mechanical-engineering#start-main-nav, https://www.aru.ac.uk/study/postgraduate/mechanical-engineering#summary, https://www.aru.ac.uk/study/postgraduate/mechanical-engineering#skip-main-nav, https://www.aru.ac.uk/study/postgraduate/mechanical-engineering'}

In [98]:
chain({"question": "Can international students work part-time while studying at the university?"}, return_only_outputs=True)

{'answer': "International students are welcome to seek paid temporary work opportunities, but there may be restrictions on working hours based on their visa. It is recommended to check with the university's Placements Officer for more information.\n",
 'sources': '\n- https://www.aru.ac.uk/arts-humanities-and-social-sciences/creative-industries/why-study-with-us/placements-and-work-experience/work-placement-faqs\n- https://www.aru.ac.uk/student-life/support-and-facilities/careers-and-employability#skip-secondary-nav\n- https://www.aru.ac.uk/student-life/support-and-facilities/careers-and-employability'}

In [99]:
chain({"question": "What are the English language proficiency requirements for admission?"}, return_only_outputs=True)

{'answer': 'The English language proficiency requirements for admission are an IELTS score of 6.0 or equivalent, with nothing lower than 5.5 in any of the four elements (listening, speaking, reading, and writing). Some courses may have higher requirements. Other accepted English language tests include ETS TOEFL iBT with 80 overall and specific minimum scores in each element, and Pearson PTE. \n',
 'sources': 'https://www.aru.ac.uk/international/entry-requirements#skip-secondary-nav, https://www.aru.ac.uk/international/entry-requirements#skip-main-nav, https://www.aru.ac.uk/international/entry-requirements, https://www.aru.ac.uk/international/entry-requirements#start-main-nav'}

In [100]:
chain({"question": "How can international students access health services while studying at ARU?"}, return_only_outputs=True)

{'answer': 'International students can access health services while studying at ARU by visiting the My ARU (student intranet) pages for international students. They can find information on registering with the health service and other relevant advice. \n',
 'sources': 'https://www.aru.ac.uk/international/planning-your-travel/before-you-arrive#skip-main-nav, https://www.aru.ac.uk/international/planning-your-travel/before-you-arrive, https://www.aru.ac.uk/international/planning-your-travel/before-you-arrive#skip-secondary-nav, https://www.aru.ac.uk/international/planning-your-travel/before-you-arrive#start-main-nav'}

In [101]:
chain({"question": "What continuing education courses does the university offer?"}, return_only_outputs=True)

{'answer': 'The university offers a short course in anaesthetic critical incidents for continuing professional development. The course is 12 weeks long and is offered through distance learning in Chelmsford.\n',
 'sources': '\n- https://www.aru.ac.uk/study/professional-and-short-courses/anaesthetic-critical-incidents\n- https://www.aru.ac.uk/study/professional-and-short-courses/anaesthetic-critical-incidents#skip-main-nav\n- https://www.aru.ac.uk/study/professional-and-short-courses/anaesthetic-critical-incidents#applynow_panel\n- https://www.aru.ac.uk/study/professional-and-short-courses/anaesthetic-critical-incidents#start-main-nav'}

In [102]:
chain({"question": "How can I apply for continuing education courses at the university?"}, return_only_outputs=True)

{'answer': "To apply for continuing education courses at the university, you can follow these steps:\n\n1. Ask your manager or Education Lead about available funding before you apply.\nSource: https://www.aru.ac.uk/study/professional-and-short-courses/principles-of-medical-and-healthcare-education#applynow_panel\n\n2. Go to UCAS Apply and fill out the application form, where you will be able to select up to five courses.\nSource: https://www.aru.ac.uk/study/undergraduate/how-to-apply#start-main-nav\n\n3. Visit the university's course search page to find the specific course you are interested in and apply accordingly.\nSource: https://www.aru.ac.uk/study/course-search?keywords=&levelofstudy=postgraduate&levelofstudy=&location=&modeofstudy=&faculty=&availability=&startdate=January&page=2\n\n",
 'sources': '- https://www.aru.ac.uk/study/professional-and-short-courses/principles-of-medical-and-healthcare-education#applynow_panel\n- https://www.aru.ac.uk/study/undergraduate/how-to-apply#sta

In [103]:
chain({"question": "What is the cost of continuing education courses?"}, return_only_outputs=True)

{'answer': 'The cost of continuing education courses is £1,583 for 30 credits (Level 7) for courses starting in 2022/23.\n',
 'sources': 'https://www.aru.ac.uk/study/professional-and-short-courses/pathophysiology-for-advanced-practice#skip-main-nav, https://www.aru.ac.uk/study/professional-and-short-courses/pathophysiology-for-advanced-practice#applynow_panel, https://www.aru.ac.uk/study/professional-and-short-courses/pathophysiology-for-advanced-practice#start-main-nav, https://www.aru.ac.uk/study/professional-and-short-courses/pathophysiology-for-advanced-practice'}

In [104]:
chain({"question": "What resources are available for continuing education students?"}, return_only_outputs=True)

{'answer': 'There is no relevant text in the given portion of the document that mentions resources available for continuing education students.\n',
 'sources': 'https://www.aru.ac.uk/anglia-learning-and-teaching/about-us#awards'}

In [105]:
chain({"question": "Can I transfer the credits earned in continuing education courses towards a degree program?"}, return_only_outputs=True)

{'answer': 'There is no information provided in the given portion of the document regarding the transfer of credits earned in continuing education courses towards a degree program.\n',
 'sources': '\n- https://www.aru.ac.uk/study/professional-and-short-courses/principles-of-medical-and-healthcare-education#applynow_panel\n- https://www.aru.ac.uk/study/course-search?categories=Education&levelofstudy=Continuing+Professional+Development'}

In [106]:
chain({"question": "How is the university reducing its carbon footprint?"}, return_only_outputs=True)

{'answer': 'The university is reducing its carbon footprint through various measures, including reducing direct carbon emissions, participating in Power Purchase Agreements for renewable energy, implementing energy-saving measures in existing buildings, and being certified to the ISO 14001 environmental management standard. They have also signed up to the UN Race to Zero initiative and publish their carbon reduction targets and progress in their Annual Report and Accounts.\n',
 'sources': '\n- https://www.aru.ac.uk/-/media/Files/about-us/sustainability/ARU-Climate-Positive-Plan.pdf\n- https://www.aru.ac.uk/-/media/Files/about-us/sustainability/working-towards-the-sustainable-development-goals-annual-report-2022.pdf\n- https://www.aru.ac.uk/about-us/sustainability/sustainable-campus/energy-carbon-and-water#skip-main-nav\n- https://www.aru.ac.uk/about-us/sustainability/sustainable-campus/energy-carbon-and-water#skip-secondary-nav'}

In [107]:
chain({"question": "How does the university promote recycling on campus?"}, return_only_outputs=True)

{'answer': 'The university promotes recycling on campus by working with the British Heart Foundation to collect unwanted items from students for reuse and recycling. They provide a Big Green Move Out Guide and campus maps on the ARU app to locate donation banks and recycling points on campus. They also emphasize the benefits of recycling and provide information on how to dispose of different items. \n',
 'sources': 'https://www.aru.ac.uk/about-us/sustainability/sustainable-campus/circular-economy#skip-main-nav'}

In [108]:
chain({"question": "What actions is the university taking to promote sustainable transportation?"}, return_only_outputs=True)

{'answer': 'The university is taking several actions to promote sustainable transportation, including collaborating with local partners to identify and promote alternatives to short-distance commuting, such as demand-responsive travel services, cycling, and e-bikes. They also prioritize ground travel, such as public transport or electric vehicles, between campuses and prioritize rail travel over air transportation for journeys less than 750 miles. Additionally, their Sustainability Policy commits them to widening the use of zero and low carbon transport and reducing the impact of commuting and business travel.\n',
 'sources': '\n- https://www.aru.ac.uk/-/media/Files/about-us/sustainability/2019-Travel_report.pdf\n- https://www.aru.ac.uk/-/media/Files/about-us/sustainability/sustainability-strategy-operations-kpi-report.pdf\n- https://www.aru.ac.uk/-/media/Files/about-us/sustainability/working-towards-the-sustainable-development-goals-annual-report-2022.pdf'}

In [109]:
chain({"question": "Has the university received any awards or recognition for its sustainability practices?"}, return_only_outputs=True)

{'answer': "Yes, the university has received awards and recognition for its sustainability practices. Some of the awards and recognition include:\n- Shortlisted for an EAUC Green Gown award for the MSc Sustainability program in partnership with Change Agents UK and the Eden Project.\n- Green Apple Award for the Young Street Health Building.\n- NUS Responsible Futures accreditation for embedding sustainability across the institution.\n- Shortlisted for a Green Gown award for the Sustainable Sainji program.\n- Finalist for Sustainable Business of the Year in the Edie Sustainability Leaders Awards.\n- Gold Green Impact award for the Students' Union.\n- Highly commended in the Sustainability Professional category at the Green Gown Awards for Professor Aled Jones.\n- International Sustainable Campus Network Award for Integration.\n- £1 million awarded by the HEFCE Revolving Green Fund for the Combined Heat and Power plant.\n- Wildlife Trust Living Landscape Award for the restoration of the 

In [110]:
chain({"question": "Are there student organizations focused on sustainability? Give one example."}, return_only_outputs=True)

{'answer': 'Yes, there are student organizations focused on sustainability. One example is the student societies that champion aspects of sustainability across all campuses. \n',
 'sources': 'https://www.aru.ac.uk/-/media/Files/about-us/sustainability/15-16078-Sustainability-Strategy-FST_final_forweb.pdf, https://www.aru.ac.uk/-/media/Files/about-us/sustainability/Society-Green-Fund-Handbook.pdf'}

In [111]:
chain({"question": "What measures does the university take to ensure student safety on campus?"}, return_only_outputs=True)

{'answer': 'The university encourages responsible behavior and asks all staff, students, and visitors to practice safe behaviors on campus. Face coverings are supported and available for free on campus. Testing before attending campus is no longer required, but individuals who are unwell with COVID-19 symptoms are advised to stay at home or leave campus. In terms of student safety on campus, the university has lockable doors, CCTV camera coverage in halls and public areas, a team of security officers available 24/7, and residential assistants providing emergency response cover. \n',
 'sources': '\n- https://www.aru.ac.uk/coronavirus/keeping-safe#skip-main-nav\n- https://www.aru.ac.uk/-/media/Files/student-life/accommodation/current-students/Management-plans-and-code-of-practice/Management-plans.pdf'}

In [112]:
chain({"question": "How can students report safety concerns or incidents?"}, return_only_outputs=True)

{'answer': 'Students can report safety concerns or incidents by using the Unsilenced Report and Support tool online or by contacting Security using the SafeZone app or by calling 01245 68 6666. \n',
 'sources': '\n- https://www.aru.ac.uk/current-students/health-and-wellbeing/emergency#start-main-nav\n- https://www.aru.ac.uk/current-students/health-and-wellbeing/emergency#suicidal-feelings\n- https://www.aru.ac.uk/current-students/health-and-wellbeing/emergency#support\n- https://www.aru.ac.uk/current-students/health-and-wellbeing/emergency#crisis'}

In [113]:
chain({"question": "What procedures does the university have in place for emergency situations?"}, return_only_outputs=True)

{'answer': "The university has the following procedures in place for emergency situations:\n- In acute or dangerous situations where a student's behavior presents an immediate risk, the Emergency Services should be contacted by dialing 999 and University Security should also be contacted on 6666.\n- After the immediate response, the Deputy Director of Student Services and relevant Dean should be notified of the details of the incident and action taken.\n- There is 24-hour security available on the Cambridge and Chelmsford campuses, and the security team can be reached at 6666 (from an ARU telephone), 01223 417734 (Cambridge), or 01245 683628 (Chelmsford).\n- In case of a genuine emergency, call 999 to contact the fire brigade, police, or ambulance service.\n- If urgent medical treatment is needed but the individual is well enough to travel, they can go to a hospital with an Accident and Emergency department (A&E), which is open 24 hours a day.\n",
 'sources': '- https://www.aru.ac.uk/-

In [114]:
chain({"question": "How does the university handle sexual assault allegations and prevention?"}, return_only_outputs=True)

{'answer': "The university has a Sexual Misconduct Policy that outlines their commitment to a supportive and victim-focused approach. They provide support, advocacy, and practical assistance to survivors of sexual violence. The university handles disclosures equally and offers both informal and formal reporting options. They respect the survivor's decision on whether or not to involve the police. There is no time limit for reporting sexual misconduct, and investigations can be conducted through the student disciplinary process or staff disciplinary procedure. Specially trained staff are involved in responding to reports of sexual misconduct. The university also promotes prevention efforts through educational programs, awareness campaigns, and bystander intervention training.\n",
 'sources': '\n- https://www.aru.ac.uk/-/media/Files/Student-Services/health-and-wellbeing/guidance-for-students-subjected-to-sexual-misconduct.pdf\n- https://www.aru.ac.uk/-/media/Files/about-us/Governance/sex

In [115]:
chain({"question": "Does the university have a crisis management plan in place?"}, return_only_outputs=True)

{'answer': 'Yes, the university has a crisis management plan in place.\n',
 'sources': '\n- https://www.aru.ac.uk/-/media/Files/about-us/Governance/AC-Minutes-180521.pdf\n- https://www.aru.ac.uk/coronavirus/studying-at-university/supporting-you-through-self-isolation/covid-19-outbreak-plan\n- https://www.aru.ac.uk/-/media/Files/about-us/Governance/FEGP-minutes-260320-APPROVED-NP.pdf'}

In [116]:
chain({"question": "What sports teams and clubs does the university have?"}, return_only_outputs=True)

{'answer': 'The university has the following sports teams and clubs: American football, Archery, Athletics, Badminton, Basketball, Cricket, Fencing, Football, Futsal, Hockey, Judo, Netball, Rowing, Rugby, Squash, Table Tennis, Taekwondo, Tennis, Ultimate Frisbee, and Volleyball.\n',
 'sources': 'https://www.aru.ac.uk/sport/sports/squash#skip-main-nav, https://www.aru.ac.uk/sport/sports-clubs/coach-and-officials-academy/becks-archibald-q-and-a, https://www.aru.ac.uk/sport/sports/squash#skip-secondary-nav, https://www.aru.ac.uk/sport/sports/squash'}

In [117]:
chain({"question": "Is there a fee for students to use the fitness center or other sports facilities?"}, return_only_outputs=True)

{'answer': 'Yes, there is a fee for students to use the fitness center or other sports facilities. The fee is £2.50 per class on a pay-as-you-go basis.\n',
 'sources': 'https://www.aru.ac.uk/sport/health-and-fitness/fitness-classes#skip-main-nav, https://www.aru.ac.uk/sport/health-and-fitness/fitness-classes#skip-secondary-nav, https://www.aru.ac.uk/sport/health-and-fitness/fitness-classes, https://www.aru.ac.uk/sport/health-and-fitness/fitness-classes#start-main-nav'}

In [118]:
chain({"question": "What types of fitness classes are available at the university fitness center?"}, return_only_outputs=True)

{'answer': 'The fitness classes available at the university fitness center include Les Mills Bodycombat, Les Mills Bodybalance, Yoga, Combat/Core, Rig Circuit, Les Mills Bodypump, Spin, Pilates, and LBT.\n',
 'sources': '\n- https://www.aru.ac.uk/sport/health-and-fitness/fitness-classes\n- https://www.aru.ac.uk/sport/health-and-fitness/fitness-classes#start-main-nav\n- https://www.aru.ac.uk/sport/health-and-fitness/fitness-classes#skip-secondary-nav\n- https://www.aru.ac.uk/sport/health-and-fitness/fitness-classes#skip-main-nav'}

In [119]:
chain({"question": "What wellness services does the university offer?"}, return_only_outputs=True)

{'answer': 'The university offers a Counselling and Wellbeing Service, which provides individual and group support, workshops, drop-in sessions, and mental health support. These services are available to all ARU students and are free and confidential. \n',
 'sources': '\n- https://www.aru.ac.uk/-/media/Files/Student-Services/health-and-wellbeing/policies/counselling-and-wellbeing-statement-of-service.pdf\n- https://www.aru.ac.uk/christmas\n- https://www.aru.ac.uk/current-students/health-and-wellbeing/counselling-and-mental-health-support/more-support#start-main-nav\n- https://www.aru.ac.uk/current-students/health-and-wellbeing/counselling-and-mental-health-support/more-support'}

In [120]:
chain({"question": "How can students get involved in sports volunteering or leadership roles on campus?"}, return_only_outputs=True)

{'answer': "Students can get involved in sports volunteering or leadership roles on campus by becoming a sports committee member or participating in the sport volunteering program offered by Anglia Ruskin Students' Union. They can speak to their club President for more information on becoming a committee member, and positions are typically elected annually at club AGMs. The sport volunteering program involves coaching, event management, and sports marketing, providing an opportunity to promote healthy living, fitness, and social inclusion in the local community. This involvement not only helps develop sporting skills but also other skills that can be used for advancement.\n",
 'sources': 'https://www.aru.ac.uk/sport/opportunities-with-us'}

In [121]:
chain({"question": "What is the university's policy on academic honesty and integrity?"}, return_only_outputs=True)

{'answer': 'The university has a policy on academic honesty and integrity. They take a formative approach to assisting students with academic honesty and provide resources such as library modules on academic skills. Statements on academic honesty are included in module guides, student handbooks, and their Academic Honesty Policy. Additionally, they require high standards of research integrity and support the UK Concordat to Support Research Integrity. Students are expected to act with honesty and integrity in their studies and assessments.\n',
 'sources': '\n- https://www.aru.ac.uk/anglia-learning-and-teaching/good-teaching-practice-and-innovation/approaches-to-learning-and-teaching/academic-honesty\n- https://www.aru.ac.uk/-/media/Files/International/MASTERCOPY-Rules-and-Regs-19th-Edition-Amended-November-2017.pdf\n- https://www.aru.ac.uk/-/media/Files/research/Guide-to-good-research-practice-LOW-RES.pdf'}

In [122]:
chain({"question": "What is the university's policy on attendance and participation in classes?"}, return_only_outputs=True)

{'answer': "There is no relevant information provided to determine the university's policy on attendance and participation in classes.\n",
 'sources': ''}

In [123]:
chain({"question": "How does the university handle disciplinary actions?"}, return_only_outputs=True)

{'answer': 'The university handles disciplinary actions by following the Student Disciplinary Procedures. If a student acts contrary to the Code of Conduct, disciplinary action may be taken by a designated ARU manager. The case is then reviewed by the Student Discipline Committee, which can approve, amend, or set aside the recommendations of the investigator. If the decision leads to exclusion or expulsion, it is notified to the Vice Chancellor for approval. The student is informed of the decision and the reasons within ten working days. If the decision does not result in exclusion or expulsion, the student has the right to appeal to the Vice Chancellor within twenty working days. If the decision does lead to exclusion or expulsion, the student has the right to appeal to the Board of Governors within twenty working days. If the student is unsatisfied with how the university handled the matter, they can raise a separate formal complaint or complain to the Office of the Independent Adjud

In [124]:
chain({"question": "Where can I find the university's code of student conduct?"}, return_only_outputs=True)

{'answer': 'The university\'s code of student conduct can be found on the Anglia Ruskin University website. It is available in the "Student Code of Conduct" document, the "Rules, Regulations and Procedures for Students (23rd Edition)" document, the "Code of Conduct for Students" document, and the "Annex to the Code of Conduct for Students" document.\n',
 'sources': '\n- https://www.aru.ac.uk/-/media/Files/International/MASTERCOPY-Rules-and-Regs-19th-Edition-Amended-November-2017.pdf\n- https://www.aru.ac.uk/-/media/Files/about-us/Governance/sexual-misconduct-policy.pdf'}

In [125]:
chain({"question": "What are the university's policies on on-campus housing and residence life?"}, return_only_outputs=True)

{'answer': "The university's policies on on-campus housing and residence life include students being responsible for keeping their rooms and communal areas clean, the majority of students living in accommodation for a maximum of one academic year, and the availability of accommodations for care leavers, students with medical conditions or disabilities, and students wishing to return. The Residential Service offices are located on campus and open from 9.30am-4.30pm. Additionally, the Residential Service manages halls of residence and houses in both Cambridge and Chelmsford campuses.\n",
 'sources': '\n- https://www.aru.ac.uk/student-life/accommodation/parent-section#skip-secondary-nav\n- https://www.aru.ac.uk/-/media/Files/student-life/accommodation/current-students/Management-plans-and-code-of-practice/Management-plans.pdf'}

In [126]:
chain({"question": "What kind of community service opportunities does the university provide?"}, return_only_outputs=True)

{'answer': 'The university provides a thriving volunteering program with hundreds of opportunities to get involved in the local community and develop skills that employers look for in graduates. Some of the community service opportunities include working on projects to create safer streets, partnering with organizations for problem-solving techniques, and making a positive social impact. These opportunities can help students develop skills such as communication, team working, and emotional intelligence.\n',
 'sources': '\n- https://www.aru.ac.uk/student-life/support-and-facilities/careers-and-employability#skip-main-nav\n- https://www.aru.ac.uk/student-life/support-and-facilities/careers-and-employability#start-main-nav\n- https://www.aru.ac.uk/student-life/support-and-facilities/careers-and-employability#skip-secondary-nav\n- https://www.aru.ac.uk/student-life/support-and-facilities/careers-and-employability'}

In [127]:
chain({"question": "How does the university promote civic engagement among students?"}, return_only_outputs=True)

{'answer': 'The university promotes civic engagement among students by promoting voting in student union elections, encouraging participation in student councils, and providing opportunities for students to get involved in campaigns. Additionally, students are expected to cooperate in the running of the university through various activities. \n',
 'sources': '\n- https://www.aru.ac.uk/-/media/Files/about-us/sustainability/Society-Green-Fund-Handbook.pdf\n- https://www.aru.ac.uk/-/media/Files/International/MASTERCOPY-Rules-and-Regs-19th-Edition-Amended-November-2017.pdf\n- https://www.aru.ac.uk/student-life/support-and-facilities/students-union-and-events#skip-secondary-nav'}

In [128]:
chain({"question": "Does the university host any events open to the community?"}, return_only_outputs=True)

{'answer': "Yes, the university hosts events open to the community. Many of these events are free and open to everyone. More information about these events can be found on the university's website and in their newsletters and email updates.\n",
 'sources': '\n- https://www.aru.ac.uk/study/open-day#skip-main-nav\n- https://www.aru.ac.uk/community-engagement#start-main-nav'}

In [129]:
chain({"question": "How does the university support sustainable practices in the community?"}, return_only_outputs=True)

{'answer': 'The university supports sustainable practices in the community by developing attractive and efficient campuses, providing opportunities for environmental learning and research, maintaining buildings and operations to meet environmental management standards, strengthening resilience against environmental risks, applying sustainability criteria in procurement, and sharing knowledge with local communities. They also engage with academic, press, NGO, business, and public sectors to have an impact on sustainability practices, build staff capacity in sustainability principles, and communicate with students about the importance of living sustainably and incorporate sustainability into curriculum development.\n',
 'sources': '\n- https://www.aru.ac.uk/-/media/Files/about-us/sustainability/15-16078-Sustainability-Strategy-FST_final_forweb.pdf\n- https://www.aru.ac.uk/-/media/Files/about-us/sustainability/Sustainability-Policy.pdf'}

In [130]:
chain({"question": "Are there any partnerships between the university and local businesses or organizations?"}, return_only_outputs=True)

{'answer': 'Yes, there are partnerships between the university and local businesses or organizations.\n',
 'sources': '\n- https://www.aru.ac.uk/community-engagement#start-main-nav\n- https://www.aru.ac.uk/graduation-and-alumni/supporting-your-university/ways-to-give#skip-main-nav\n- https://www.aru.ac.uk/graduation-and-alumni/supporting-your-university/ways-to-give#start-main-nav\n- https://www.aru.ac.uk/graduation-and-alumni/supporting-your-university/ways-to-give'}

In [131]:
chain({"question": "What research fields are the university's science and engineering faculty members currently focusing on?"}, return_only_outputs=True)

{'answer': "The university's science and engineering faculty members are currently focusing on research fields such as animal and environmental biology, societies and groups, psychology, sport and exercise sciences, mind and behavior, cybersecurity and networking, health, social care and medical innovation, computer science, computing and technology, artificial intelligence, consumption and change, vision and eye research, life sciences, global risk and resilience, cell and molecular biology, forensic science and crime, sustainability, microbiology, cancer biology, molecular techniques, education for sustainability, neuroscience, electrical and electronic engineering, policing, molecular diagnostics, and anatomy and physiology.\n",
 'sources': '\n- https://www.aru.ac.uk/science-and-engineering/faculty-staff?location=Cambridge&faculty=Science+and+Engineering\n- https://www.aru.ac.uk/science-and-engineering/research/applying-for-research-degree#start-main-nav'}

In [132]:
chain({"question": "How can students get involved in research projects at the university?"}, return_only_outputs=True)

{'answer': "Students can get involved in research projects at the university by exploring the project opportunities identified by the research staff. These projects offer a wide range of opportunities for students to develop their own projects and pursue their passion for learning. Additionally, there is a summer placement scheme available where students can work alongside academics to produce world-class research. More information about these opportunities can be found on the university's website.\n",
 'sources': '\n- https://www.aru.ac.uk/arts-humanities-and-social-sciences/research/postgraduate-research/application-tips\n- https://www.aru.ac.uk/science-and-engineering/research/research-project-opportunities#start-main-nav\n- https://www.aru.ac.uk/business-and-law/research/about#skip-main-nav'}

In [133]:
chain({"question": "Can you provide information about the recent major research breakthroughs at the university?"}, return_only_outputs=True)

{'answer': 'There is no information provided in the given portion of the document about recent major research breakthroughs at the university.\n',
 'sources': 'https://www.aru.ac.uk/-/media/Files/Faculty/FSE/Research/First---Summer-2022.pdf, https://www.aru.ac.uk/study/explore-aru/25-ways-aru-has-made-a-difference#start-main-nav, https://www.aru.ac.uk/-/media/Files/Faculty/FSE/Research/First-newsletter---Spring-2020v2.pdf, https://www.aru.ac.uk/study/explore-aru/25-ways-aru-has-made-a-difference#skip-secondary-nav'}

In [134]:
chain({"question": "What resources does the university provide to support student research?"}, return_only_outputs=True)

{'answer': 'The university provides several resources to support student research, including the Resources team that assists with purchasing consumables and equipment, travel and expenses, finance administration, and resources administration. The university also has a library available for students. Additionally, students are allocated supervisors and have access to professional support from the publishing industry. There are research events, seminars, and conferences for students to participate in, and financial support is available for travel and conference expenses. The university also offers support for grant applications, knowledge transfer initiatives, and career development through the Research and Innovation Development Office and the Doctoral School.\n',
 'sources': '\n- https://www.aru.ac.uk/-/media/Files/Faculty/FSE/Research/First---Summer-2020V3.pdf\n- https://www.aru.ac.uk/-/media/Files/International/pre-departure-guide-2021-22-arriving-in-the-uk.pdf\n- https://www.aru.ac.

In [135]:
chain({"question": "How does the university collaborate with other institutions or companies on research projects?"}, return_only_outputs=True)

{'answer': 'The university collaborates with other institutions or companies on research projects through collaborative research partnerships. These partnerships involve two or more parties bringing their own expertise to the project, and they are often funded by research institutes, government agencies, or charities. The results of the research are published, and Anglia Ruskin University retains ownership of the intellectual property.\n',
 'sources': '\n- https://www.aru.ac.uk/science-and-engineering/research/research-services#skip-main-nav\n- https://www.aru.ac.uk/science-and-engineering/research/research-services\n- https://www.aru.ac.uk/science-and-engineering/research/research-services#skip-secondary-nav\n- https://www.aru.ac.uk/science-and-engineering/research/research-services#start-main-nav'}